**Manisa Celal Bayar Üniversitesi - Yazılım Mühendisliği Bölümü - YZM-3226 Makine Öğrenmesi Dersi -  Ödev#1**

* **152804001 - Utku Eren YILMAZ**
* **152804012 - Cengiz CEBECİ**



Makine Öğrenmesi dersi ödevinin ikinci kısmı için seçtiğimiz DataSetinde geçmişten günümüze kadar farklı platformdaki video-oyunlarının bilgileri bulunmaktadır.

* Bu veri seti, oyunların satış oranları ile yakaladıkları başarı sıralamaları arasında düzgün bir bağlantının kurulmuş olması ile bize korelasyon anlamında kolaylık sağlayacağı düşünüldüğü için seçilmiştir.
* Ödevdeki amacımız elimizdeki veri setindeki video-oyunlarından yalnızca Bilgisayar platformu için üretilen oyunlar kalacak şekilde soyutlayarak, bu oyunların satış oranları ile tüm oyunlar arasında nasıl bir başarı puanı aldığını inceleyebilmektir.

İlk olarak seçtiğimiz veri setini inceliyoruz ve genel bir veri keşfi gerçekleştiriyoruz:

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn import metrics

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import model_selection
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score



#Datamızın ilk 5 satırını inceliyoruz.
df = pd.read_csv("../input/vgsales.csv")
df.head() #İlk 5 satırı inceliyoruz

In [ ]:
#Son 5 satırını inceliyoruz
df.tail()

In [ ]:
#İstatiksel değerlerine bakıyoruz
df.describe()

In [ ]:
#Sütunların veritipleri inceleniyor
df.info()

In [ ]:
#Dataset içerisindeki toplam satır ve sütun sayıları gösteriliyor
df.shape

Bu noktada yaptığımız genel incelemelere göre 11 sütun içerisinde toplamda 16598 farklı veri olduğunu tespit ediyoruz.
Şimdi ise ödev içeriğini Türkçeleştirmek ve bazı değerleri normalleştirmek için işlemler uyguluyoruz.

In [ ]:
#Sütun isimleri Türkçeleştiriliyor.
df = df.rename(columns={'Rank': 'BasariSiralamasi', 
                        'Name': 'OyunAdi', 
                        'Year' : 'CikisTarihi', 
                        'Genre':'OyunTarzi',
                        'Publisher': 'Yayimci',
                        'NA_Sales': 'KuzeyAmerikaSatis',
                       'EU_Sales' : 'AvrupaSatis',
                       'JP_Sales' : 'JaponyaSatis',
                       'Other_Sales' : 'DigerSatis',
                        'Global_Sales' : 'GlobalSatis'})

In [ ]:
#Satış değerleri içerisindeki veriler ondalıklı halde bulunduklarından bu değerleri normalleştiriyoruz

df['KuzeyAmerikaSatis'] = df['KuzeyAmerikaSatis'] * 100
df['AvrupaSatis'] = df['AvrupaSatis'] * 100
df['JaponyaSatis'] = df['JaponyaSatis'] * 100
df['DigerSatis'] = df['DigerSatis'] * 100
df['GlobalSatis'] = df['GlobalSatis'] * 100

df.head()

Bu noktada elimizdeki KuzeyAmerikaSatis, AvrupaSatis, JaponyaSatis ve DigerSatis sütunlarının 
toplamları ile GlobalSatis sütunu zaten elde edildiğinden dolayı bizim bu kayıtları kullanarak
****yeni bir toplam özniteliği çıkarmamıza gerek kalmıyor.****

Global verilerle ilgileneceğimiz için bölgesel satış değerlerini tablomuzdan çıkartıyoruz.

In [ ]:
#Gereksiz sütunların çıkarılması işlemi gerçekleştiriliyor ve 3 rastgele örnek görüntüleniyor.
df = df[['BasariSiralamasi', 'OyunAdi','Platform','CikisTarihi','OyunTarzi','Yayimci','GlobalSatis']]
df.sample(3)

 Buradan sonra bizim amacımız platform olarak yalnızca bilgisayar oyunlarının seçilmesi şeklinde devam edecek ve bu doğrultuda verileri inceleyeceğiz.

In [ ]:
#Platformu PC olan veriler seçilerek tablo güncelleniyor
df = df[(df['Platform']=='PC')]
#Oluşan yeni satır-sütun sayılarının görülmesi
df.shape

* Tüm veriler içerisinde 960 adet bilgisayar oyunu olduğunu görüyoruz.
* Şimdi elimizdeki sütunlar içerisindeki kayıtlarda girilmemiş - boş değerler olup olmadığını kontrol ediyoruz.

In [ ]:
#Tüm boş değerleri bulup toplamlarının yazdırılması
df.isnull().sum()

Çıkış Tarihi ve Yayimci içerisinde  girilmemiş değerler olduğunu tespit ettik. Bu değerleri herhangi bir yöntem kullanarak doldurmadan veya silmeden önce filtreleme işlemlerini gerçekleştirip sonra tekrar kontrol edeceğiz.
Sayısal değerlerin dağılımlarını incelemek için histogram grafikleri ile inceleme yapıyoruz.

In [ ]:
#Büyük ölçekte dağılımların görülmesi için ayrı ayrı yazılıyor.
df.hist('BasariSiralamasi')
df.hist('CikisTarihi')
df.hist('GlobalSatis')

Histogramlarda Başarı Sıralamalarının iyi seviyede bir dağılım gösteridiği görülürken Çıkış Tarihlerinin ve Global Satış oranlarının orantısız bir dağılım gösterdiğini görüyoruz.
* Bu noktada bu değerler için belirli filtrelemeler gerçekleştiriliyor.


İlk olarak incelediğimiz oyunların daha modern oyun teknolojilerinin kullanılmaya başlandığı zamanlar arasında olmasını sağlamak için 2000 yılından günümüze kadar çıkan oyunlar olarak filtreleme yapıyoruz.

In [ ]:
#CikisTarihi için filtreleme yapılıyor.
df = df[(df['CikisTarihi']>2000) & (df['CikisTarihi']<2019)] 

df.hist('CikisTarihi')

Yaptığımız filtreleme işlemi ile Çıkış Tarihlerinin eskisine göre daha normal-eşit bir dağılım gösterdiğini görüyoruz.
Ayrıca aykırı değerler olup olmadığını yine de görüntüleniyor.

In [ ]:
#Uç değerler inceleniyor
sns.boxplot(x=df['CikisTarihi'])

Çıkış tarihi filtrelememizin ardından Global satış değerlerimizde meydana gelen yeni dağılımı tekrar görüntülüyoruz.

In [ ]:
df.hist('GlobalSatis')

Bu noktada satış değerlerinin hala aşırı derecede sol tarafa yığıldığını görüyoruz ve aykırı değerleri inceleyerek sırayla filtrelemeler gerçekleştiriyoruz.

In [ ]:
sns.boxplot(x=df['GlobalSatis'])

In [ ]:
#İlk filtreleme
df = df[(df['GlobalSatis']>10) & (df['GlobalSatis']<250)] 
sns.boxplot(x=df['GlobalSatis'])
df.hist('GlobalSatis')

İlk filtreleme ardından verinin hala dengeli dağılmadığını görüyoruz bu noktada elimizde kalan veri sayısını tespit etmek için tekrar bir inceleme gerçekleştiriyoruz ve sonrasında filtreleme işlemine devam ediyoruz. Bu sayede yaptığımız filtreleme sonucunda çok veri kaybedip kaydetmediğimizi tespit etmiş olacağız.

In [ ]:
df.shape

Bu noktada elimizde 256 adet veri kalmış durumda, tekrar bir filtreleme yapıp eğer elimizdeki veri çok azalırsa buradan devam etmeyi planlıyoruz.

In [ ]:
#İkinci filtreleme
df = df[(df['GlobalSatis']<100)] 
sns.boxplot(x=df['GlobalSatis'])
df.hist('GlobalSatis')

In [ ]:
df.shape

Yaptığımız filtreleme işleminden sonra hala elimizde yeterli çoklukta veri olduğu görüldüğünden bu kayıtlar ile devam ediyoruz.

* Bu noktada Satış değerlerimiz eskisine göre daha normal bir dağılım göstermeye başlıyor ve hala elimizde çok sayıda veri mevcut.
* Veri dağılımlarımızı şimdilik daha eşit hale getirmiş olduk, şimdi tekrar boş kayıtlar olup olmadığını kontrol ediyoruz.

In [ ]:
df.isnull().sum()

* Daha önce bulmuş olduğumuz boş Çıkış Tarihi değerlerin gittiğini ve Yayimci sütunundaki boş değerlerin ise 2ye düştüğünü görüyoruz.
* Gerçekleştirmeyi planladığımız ödev içerisinde Oyun Tarzı ve Yayımcı gibi özelliklere ihtiyaç duymadığımızdan dolayı Yayımcı içerisindeki boş değerleri doldurmak yerine tablomuzu  bu iki sütundan da soyutluyoruz.

In [ ]:
#Tabloda yalnızca kullandığımız sütunlar bırakılıyor.
df = df[['BasariSiralamasi', 'OyunAdi','Platform','CikisTarihi','GlobalSatis']]

Verilerimizi çıkış tarihine göre sıralayarak tekrar göz atıyoruz.

In [ ]:
#Sıralama işlemi gerçekleştiriliyor.
df = df.sort_values('CikisTarihi', axis=0, ascending=True)
df.head(10)

Bu noktada elimizdeki tüm veriler ile oluşan korelasyon değerlerini sayısal ve ısı haritası olarak inceliyoruz.

In [ ]:
#İlişkilerin sayısal incelemesi
df.corr()

In [ ]:
#Isı haritası
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

Planladığımız gibi Global Satış oranları ile Oyunun başarı sıralaması arasında yüksek bir bağlantı olduğunu görüyoruz

In [ ]:
#Bulunan ilişkinin çizim işlemi gerçekleştiriliyor.
df.plot(x='GlobalSatis', y='BasariSiralamasi', style='.')  
plt.title('Global Satış Derecesi - Başarı Sıralaması')  
plt.xlabel('Global Satışlar')  
plt.ylabel('Başarı Sıralamaları')  
plt.show() 

Veri keşfimizi tamamladıktan sonra yüksek korelasyona sahip bu iki sütunu kullanarak 2 adet öğrenme modeli uygulayıp bunların başarılarını inceleyeceğiz.

In [ ]:
x = df.iloc[:,4].values.reshape(-1,1) #GlobalSatis sütunundaki değerler x 
y = df.iloc[:,0].values.reshape(-1,1) #BaşarıSiralamasi sütunundaki değerler y

Şimdi elimizdeki x ve y sütunlarını test ve eğitim kümeleri olarak ayırıyoruz.
* Test kümesi %30 olarak belirleniyor.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

****İlk olarak Lineer Regresyon modelini kullanarak işlemlerimizi gerçekleştirecek ve bu modelin başarısını inceleyeceğiz.****

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

# Lineer Regresyon Modeli Eğitimi
lr.fit(x_train, y_train.ravel())

#Modelin yaptığı tahminler y_pred olarak tanımlanıyor.
y_pred = lr.predict(x_test)

* Oluşan lineer eğrinin özelliklerine göz atıyoruz

In [ ]:
print("Kesim Noktası: ",lr.intercept_)
print("Eğim: ",lr.coef_)

Lineer Regresyon modeli ile gerçekleştirilen eğitim sonucunda elde edilen tahminsel değerler ile bilinen gerçek sonuçları tablo üzerinde inceliyor-karşılaştırıyoruz.

In [ ]:
#Test verilerinin ve Tahminlerin tablosal olarak incelenmesi
LRegressionTablo = pd.DataFrame({'Gerçek':y_test.ravel(),'Tahmin':y_pred.ravel()})
LRegressionTablo

In [ ]:
#Tahminler ve gerçek değerler çizdiriliyor.
plt.scatter(x_test, y_test, color='red')
plt.plot(x_test, y_pred ,color="blue")
plt.title('Lineer Regresyon Doğrusu')
plt.xlabel('Global Satislar')
plt.ylabel('Basari Siralamasi')
plt.show()

Kullandığımız lineer regresyon algoritmasının başarı ölçütleri olan MSE ve RMSE değerlerini inceliyoruz.

In [ ]:
#Regresyon için başarı ölçütleri
print('Mean Absolute Error: ',metrics.mean_absolute_error(y_test,y_pred))
print('Mean Squared Error: ',metrics.mean_squared_error(y_test,y_pred))
print('Root Mean Squared Error: ',np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

Lineer Regresyon algoritmasının genel başarı durumunu görmek için ise Score'una göz atıyoruz.

In [ ]:
#Başarı ölçütü
print('Lineer Regresyon Başarısı: ',r2_score(y_test, y_pred))

Lineer Regresyon Algoritmasının %87 lik bir başarıya ulaşabildiği görülüyor.

Şimdi bir başka öğrenme modeli uygulayarak daha yüksek bir başarı yakalayıp yakalayamayacağımız incelenecek.


****Şimdi Naive Bayes algoritmasını kullanacağız.****
* İlk olarak test ve eğitim kümeleri farklı değerler alması için tekrar seçiliyor.


In [ ]:
#CM için..
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(x, y, test_size=validation_size, random_state=seed)

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

scoring = 'accuracy'
kfold = model_selection.KFold(n_splits=10, random_state=seed) #KFold model tercih ediliyor.
cv_results = model_selection.cross_val_score(nb, X_train, Y_train, cv=kfold, scoring=scoring)

x_train2, x_test2, y_train2, y_test2 = train_test_split(x, y, test_size=0.3, random_state=0)

In [ ]:
# Naive Bayes - Gaussian Naive Bayes Modeli Eğitimi
nb.fit(x_train2, y_train2)
#Modelin yaptığı tahminler y_pred olarak tanımlanıyor.
y_pred2 = nb.predict(x_test2)

Elde edilen tahminsel değerler ile bilinen gerçek sonuçları tablo üzerinde inceliyor-karşılaştırıyoruz.

In [ ]:
NBayesTablo = pd.DataFrame({'Gerçek':y_test2.ravel(),'Tahmin':y_pred2.ravel()})
NBayesTablo

In [ ]:
#NB için tahminler ve gerçek değerler çizdiriliyor.
plt.scatter(x_test2, y_test2, color='red')
plt.plot(x_test2, y_pred2 ,'b+')
plt.title('Naive Bayes Algoritması Tahminleri')
plt.xlabel('Global Satislar')
plt.ylabel('Basari Siralamasi')
plt.show()

Yaptığımız incelemede kabaca Lineer Regresyon'daki tahminlere oranla Naive Bayes kullanıldığında daha yakın tahminler bulunduğunu görüyoruz.

Lineer Regresyon modelini MSE ve RMSE ölçütleri ile incelemiştik. Naive Bayes'de ise Karmaşıklık matrisi, ACC, Precision, Recall kullanıyoruz.

In [ ]:
# NB öltütleri inceleniyor
print(classification_report(y_test2, y_pred2))
print(confusion_matrix(y_test2, y_pred2))
# Accuracy score
from sklearn.metrics import accuracy_score
print("ACC: ",accuracy_score(y_pred2,y_test2))

Naive Bayes algoritmasının genel başarı durumunu görmek için ise Score'una göz atıyoruz.

In [ ]:
print('Naive Bayes Başarısı: ',r2_score(y_test2, y_pred2))

Naive Bayes Algoritmasının %98 lik bir başarıya ulaşabildiği görülüyor.

**SONUÇ**
* Kullandığımız iki öğrenme modeli olan Lineer Regresyon ve Naive Bayes Algoritmalarında aldığımız sonuçları tablosal ve görsel olarak incelediğimizde, Başarı derecelerini karşılaştırdığımızda Naive Bayes - Gaussian NB algoritmasının veri setimizin eğitimi için daha uygun bi model olduğu sonucuna varıyoruz.
*  Naive Bayes ile %98 başarılı sonuçlar alabilirken bu oran Lineer Regresyonda %87lerde kalmakta.



**ÖDEVE EK İNCELEMELER**

Farklı 2 modele daha genel olarak göz attık:

* Decision Tree (Karar Ağacı) modeli de uygulasaydık alacağı başarı puanını görelim.

In [ ]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(x, y, test_size=0.3, random_state=0)
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth = 5)

dtr.fit(x_train3, y_train3)
y_pred3 = dtr.predict(x_test3)

plt.scatter(x_test3, y_test3, color='red')
plt.plot(x_test3, y_pred3 ,'b+')
plt.title('Decision Tree Algoritması Tahminleri')
plt.xlabel('Global Satislar')
plt.ylabel('Basari Siralamasi')
plt.show()

print('Başarı Puanı: ',r2_score(y_test3, y_pred3))

Karar Ağaçlarının ise %99 başarı puanı alabileceğini görmüş olduk.

In [ ]:
x_train4, x_test4, y_train4, y_test4 = train_test_split(x, y, test_size=0.3, random_state=0)

from sklearn.preprocessing import PolynomialFeatures
pr = PolynomialFeatures(degree = 2)

x_poly = pr.fit_transform(x_train4)

# Lineer Regresyon Modeli Eğitimi
pr.fit(x_poly, y_train4)

lr2 = LinearRegression()

# Lineer Regresyon Modeli Eğitimi
lr2.fit(x_poly, y_train4.ravel())

#Modelin yaptığı tahminler y_pred olarak tanımlanıyor.
y_pred4 = lr2.predict(pr.fit_transform(x_test4))


#Test verilerinin ve Tahminlerin tablosal olarak incelenmesi
PRegressionTablo = pd.DataFrame({'Gerçek':y_test4.ravel(),'Tahmin':y_pred4.ravel()})
PRegressionTablo.head()

In [ ]:
plt.scatter(x_test4, y_test4, color='red')
plt.scatter(x_test4, y_pred4.ravel() ,color="blue")
plt.title('Polinomsal Regresyon Doğrusu')
plt.xlabel('Global Satislar')
plt.ylabel('Basari Siralamasi')
plt.show()

In [ ]:
print('Mean Absolute Error: ',metrics.mean_absolute_error(y_test4,y_pred4))
print('Mean Squared Error: ',metrics.mean_squared_error(y_test4,y_pred4))
print('Root Mean Squared Error: ',np.sqrt(metrics.mean_squared_error(y_test4,y_pred4)))

print('Polinomsal Regresyon Başarısı: ',r2_score(y_test4, y_pred4))

**SONUÇ**

Uygulanan 4 farklı modelde sağlanan başarılar:
* Lineer Regresyon: %87
* Naive Bayes: %99
* Decision Tree: %99
* Polinomsal Regresyon: %98 